In [1]:
from andreiNet.neural_net import NeuralNetwork, norm_data, one_hot_encode

In [2]:
# Load Iris Dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import time

In [3]:
iris = datasets.load_iris()
X = iris.data  
y = iris.target


feature_idxs = [1, 3] # SET FEATURES BY INDEX <------------------

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
xlbl, ylbl = feature_names[feature_idxs[0]], feature_names[feature_idxs[1]] 
# We will also split the dataset into training and testing so we can evaluate the kNN classifier
X_trn_, X_test_, y_trn, y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.333, 
                                                 random_state=0,
                                                 stratify=y)
X_trn, X_test = X_trn_[:, feature_idxs], X_test_[:, feature_idxs]

print("X_trn.shape = {}, X_test.shape = {}".format(X_trn.shape, X_test.shape))
X_trn_norm, (trn_mean, trn_std) = norm_data(X_trn)
X_test_norm = (X_test - trn_mean) / trn_std



X_trn.shape = (100, 2), X_test.shape = (50, 2)


In [4]:
activation = 'ReLU'
batch_size = 50
random_state = 0
lr = 0.1
n_epochs = 100

start_time = time.time()
nn = NeuralNetwork(hidden=(8,), 
                   init_weights='he_norm', 
                   activation=activation,
                   shuffle=True,
                   batch_size=batch_size,
                   random_state=random_state
                   )
nn.train(X_trn_norm, y_trn, n_epochs=n_epochs, lr=lr)
y_pred_trn = nn.predict(X_trn_norm).argmax(axis=1)
y_pred_test = nn.predict(X_test_norm).argmax(axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

print('trn acc', accuracy_score(y_pred_trn, y_trn))
print('test acc', accuracy_score(y_pred_test, y_test))

epoch 100: final trn loss = 0.34065946969420763
--- 0.04808950424194336 seconds ---
trn acc 0.97
test acc 0.94


In [5]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

np.random.seed(random_state)
start_time = time.time()

model = Sequential()
model.add(Dense(8, activation=activation.lower(), 
                input_dim=X_trn_norm.shape[1], 
                kernel_initializer='he_normal'))
model.add(Dense(3, activation='softmax',))

sgd = SGD(lr=lr, decay=0.0, momentum=0.00, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_trn_norm, one_hot_encode(y_trn, 3),
          epochs=n_epochs,
          batch_size=batch_size,
          verbose=0)
y_pred_trn = model.predict(X_trn_norm).argmax(axis=1)
y_pred_test = model.predict(X_test_norm).argmax(axis=1)
print("--- %s seconds ---" % (time.time() - start_time))
print('trn acc', accuracy_score(y_pred_trn, y_trn))
print('test acc', accuracy_score(y_pred_test, y_test))

Using TensorFlow backend.


--- 0.4775118827819824 seconds ---
trn acc 0.97
test acc 0.92
